從《Python 網路爬蟲與資料分析入門實戰》第三章的範例中練習爬蟲

從第三章奇摩電影爬蟲範例中練習自己寫的程式碼：爬蟲上映中的電影

書中原始程式碼來源：https://github.com/jwlin/web-crawler-tutorial/tree/master/ch3

# 上映的電影：(1)爬取一頁函數(2)爬取多頁函數(3)處理資料並儲存

## 1-1爬取一頁資料函數：將上映中的電影抓下來(程式碼同本周新片)

In [1]:
import requests
import re
from bs4 import BeautifulSoup

url='https://movies.yahoo.com.tw/movie_intheaters.html'

def yahoo_movie(url):
    #1.與網頁溝通/連線(使用requests套件)
    resp = requests.get(url)
    #2.剖析網頁原始碼(使用BeautifulSoup剖析器) 
    soup = BeautifulSoup(resp.text, 'html5lib')
    #3.取得網頁內容：3.1電影主資訊的位置
    rows = soup.find_all('div', 'release_info_text')
        #3.2.創造空清單變數存放所有資訊
    movies = []
        #3.3.迴圈跑電影資訊的內容
    for row in rows:
        #3.4.創造空的字典存放每個電影資訊
        movie = dict()
        #3.5.創造中文電影名稱存放電影的「中文名稱」:div標籤/release_movie_name屬性/a標籤
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        #3.5.創造「英文名稱」:div標籤/release_movie_name屬性中的div標籤/en屬性/a標籤
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
            ##3.5.1.創造符合上映日期的格式(三組一或多個數字)
        pattern = '\d+-\d+-\d+'
            ##3.5.2.將上映日期位置(div標籤/release_movie_time屬性)的內容放入匹配格式的re.search套件中
        match = re.search(pattern,row.find('div', 'release_movie_time').text)
            ##3.5.2創造「上映日期」:將符合格式的內容放入
        movie['release_date'] = match.group(0)
        #3.6創造「期待值」:div標籤/leveltext屬性/span標籤
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        #3.7創造「電影網址」:div標籤/release_movie_name屬性/a標籤的href
        movie['url'] = row.find('div', 'release_movie_name').a['href']
        #3.8創造「電影ID」:從電影網址中去除.html,再分隔-取出最右邊的電影ID
        movie['movie_id'] =movie['url'].split('.html')[0].split('-')[-1]
        #3.9創造「預告網址」:主資訊的父節點的前一個兄弟節點的div標籤/release_foto屬性/a標籤的img的src
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        #3.10t創造「簡介」:div標籤/release_text屬性並取代\n
        movie['intro'] = row.find('div', 'release_text').text.replace('\n', '').strip()
        #3.11創造變項trailer_a放入預告片網址位置:div標籤/release_btn color_btnbox屬性的所有第二個a標籤
        trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
            #3.11.1創造「預告片網址」：如果href有在變項中就放入變項的href內容，沒有則填入空白
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        #3.12將上述資料添加到movies中
        movies.append(movie)
        #3.13返回所有電影資訊的變項
    return movies
yahoo_movie(url)

[{'ch_name': '小小夜曲',
  'eng_name': 'Little Nights, Little Love',
  'expectation': '61%',
  'intro': '這是一段由平凡的你我，創造的小小奇蹟故事！仙台車站前人來人往的天橋上，人們聚精會神地看著戶外電視牆上播送的世界重量級拳王爭霸戰，同一時間，27歲的單身上班族佐藤（三浦春馬 飾），則在一旁焦頭爛額地作著問卷調查，正當他屢屢遭到路人拒絕，準備放棄時，卻受到旁邊街頭歌手的歌聲吸引，在他專注聆聽的同時，也注意到另一個和他一樣停下腳步的身影，是一名穿著黑色套裝的女子，他鼓起勇氣向女子詢問是否能接受問卷調查，在女子填寫的同時，佐藤注意到女子手上寫著“洗髮精”三個字，此後，這三個字像魔咒般烙印在他的腦海中，總是期待著再次與這名手上寫著“洗髮精”的女子相遇，就連在超市見到洗髮精，也會想起她的身影…。\xa0與此同時，佐藤身邊人們的經歷，也一再讓他思索著「真的有所謂的命中注定這一回事嗎？那麼要如何才能與命中注定的人相遇？」，公司前輩與妻子之間愛的開始與破滅；大學死黨與班花看似不配，卻意外合拍的婚姻；好友口中的美容師朋友與客人的弟弟之間，「只聞其聲，不見其人」的曖昧戀情。一段段看似無關，卻又環環相扣的關係橫跨十年歲月，一場笑淚交織、充滿奇蹟的愛情冒險，就此展開！\xa0日本《達．文西》雜誌票選最受歡迎男作家No.1！伊坂幸太郎筆下唯一純愛作品。這本書沒有推理嫌犯，也沒有英雄出場喔，主演的人，就是你──由平凡的你我創造的小小奇蹟故事！結合伊坂幸太郎特有的幽默與情感，充滿伏筆和驚喜的療癒作品！看完之後，你必定會期待明天的來臨。',
  'movie_id': '10213',
  'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/September2019/aSPz9kYoN3Gyg4Rcj0ea-2751x3908.jpg',
  'release_date': '2019-11-22',
  'trailer_url': 'https://movies.yahoo.com.tw/video/%E5%B0%8F%E5%B0%8F%E5%A4%9C%E6%9B%B2-%E4%B8%AD%E6%9

## 1-2爬取多頁資料函數：自動重複爬取資料

In [2]:
url='https://movies.yahoo.com.tw/movie_intheaters.html'

#4.自動儲存所有上映中的電影內容
def repeat_yahoo_movie(url):
    #4.1創造空清單變數存放所有資訊
    movies=[]
    #4.2用函數爬出第一頁內容
    movie=yahoo_movie(url)
    #4.3將第一頁內容加入變數中
    movies += movie
    #4.4向第一頁搜尋網頁發出請求
    resp = requests.get(url)
    #4.5剖析原始碼
    soup = BeautifulSoup(resp.text, 'html5lib')
    #4.6如果有下一頁則創造下一頁的網址變數,沒有則創造空清單
    if soup.find('li', 'nexttxt').a['href']:
        nextpage = soup.find('li', 'nexttxt').a['href']
    else:
        nextpage = []
    #4.7如果有下一頁的網址(若下一頁不等於空清單的迴圈)則爬新的內容加入變數中
    while nextpage!=[]:
        #4.7.1將下一頁內容爬出來
        new_movie=yahoo_movie(nextpage)
        #4.7.2將新內容加入變數中
        movies += new_movie
        #4.7.3向下一頁的網址發出請求
        resp = requests.get(nextpage)
        #4.7.4剖析下一頁的原始碼
        soup = BeautifulSoup(resp.text, 'html5lib')
        #4.7.5若仍有下下頁的網址則創造新的下一頁網址變數,沒有則創造空清單
        if soup.find('li', 'nexttxt').a:
            nextpage = soup.find('li', 'nexttxt').a['href']
        else:
            nextpage = []
    #4.8返回所有電影資訊
    return movies
        
repeat_yahoo_movie(url)

[{'ch_name': '小小夜曲',
  'eng_name': 'Little Nights, Little Love',
  'expectation': '61%',
  'intro': '這是一段由平凡的你我，創造的小小奇蹟故事！仙台車站前人來人往的天橋上，人們聚精會神地看著戶外電視牆上播送的世界重量級拳王爭霸戰，同一時間，27歲的單身上班族佐藤（三浦春馬 飾），則在一旁焦頭爛額地作著問卷調查，正當他屢屢遭到路人拒絕，準備放棄時，卻受到旁邊街頭歌手的歌聲吸引，在他專注聆聽的同時，也注意到另一個和他一樣停下腳步的身影，是一名穿著黑色套裝的女子，他鼓起勇氣向女子詢問是否能接受問卷調查，在女子填寫的同時，佐藤注意到女子手上寫著“洗髮精”三個字，此後，這三個字像魔咒般烙印在他的腦海中，總是期待著再次與這名手上寫著“洗髮精”的女子相遇，就連在超市見到洗髮精，也會想起她的身影…。\xa0與此同時，佐藤身邊人們的經歷，也一再讓他思索著「真的有所謂的命中注定這一回事嗎？那麼要如何才能與命中注定的人相遇？」，公司前輩與妻子之間愛的開始與破滅；大學死黨與班花看似不配，卻意外合拍的婚姻；好友口中的美容師朋友與客人的弟弟之間，「只聞其聲，不見其人」的曖昧戀情。一段段看似無關，卻又環環相扣的關係橫跨十年歲月，一場笑淚交織、充滿奇蹟的愛情冒險，就此展開！\xa0日本《達．文西》雜誌票選最受歡迎男作家No.1！伊坂幸太郎筆下唯一純愛作品。這本書沒有推理嫌犯，也沒有英雄出場喔，主演的人，就是你──由平凡的你我創造的小小奇蹟故事！結合伊坂幸太郎特有的幽默與情感，充滿伏筆和驚喜的療癒作品！看完之後，你必定會期待明天的來臨。',
  'movie_id': '10213',
  'poster_url': 'https://movies.yahoo.com.tw/x/r/w420/i/o/production/movies/September2019/aSPz9kYoN3Gyg4Rcj0ea-2751x3908.jpg',
  'release_date': '2019-11-22',
  'trailer_url': 'https://movies.yahoo.com.tw/video/%E5%B0%8F%E5%B0%8F%E5%A4%9C%E6%9B%B2-%E4%B8%AD%E6%9

## 1-3 處理資料成dataframe：印出資料筆數與期待值高的電影並儲存csv檔案

In [3]:
import pandas as pd
import numpy as np

#5.將上映的電影資料整理成dataframe,印出符合條件的電影並儲存成csv檔
    #5.1用函數取出所有資料
data=repeat_yahoo_movie(url)
    #5.2命名各個欄位順序(不命名則會照開頭字母順序)
name=['ch_name','eng_name','release_date','expectation','url','movie_id','poster_url','intro','trailer_url']
    #5.3將資料轉成dataframe
test=pd.DataFrame(columns=name,data=data)
    #5.4整理資料中的索引(從1開始編號)
test.index = np.arange(1,len(test)+1)
    #5.5將索引重新命名成電影編號
test.index.names = ['movie_NO.']
    #5.6顯示資料量(共有幾部電影)
print('目前上映', len(data), '部電影')
    #5.7顯示期待值超過90%的電影名稱
print('期待值> 90% :' )
n='90%'
for a in data:
    if a['expectation'] > n :
        print(a['ch_name'],a['expectation'])
        
    #5.8印出所有電影
print('上映中的電影:',test)
    #5.9儲存成csv檔案
test.to_csv('newmovie.csv',encoding='utf_8_sig')

目前上映 89 部電影
期待值> 90% :
冰雪奇緣2 92%
神機有毛病 95%
萬萬沒想到 97%
催眠．裁決 96%
安眠醫生 95%
遺怨 92%
黑金叛徒 91%
魔鬼終結者：黑暗宿命 95%
傻傻愛你，傻傻愛我 94%
陽光普照 94%
紫羅蘭永恆花園外傳－永遠與自動手記人偶－ 97%
陪你很久很久 94%
黑魔女2 96%
江湖無難事 95%
海獸之子 97%
小丑 98%
返校 95%
星際救援 95%
花椒之味 96%
大叔之愛電影版 96%
牠 第二章 97%
第九分局 93%
全面攻佔3：天使救援 99%
玩命關頭：特別行動 94%
獅子王 94%
蜘蛛人：離家日 95%
寄生上流 94%
玩具總動員4 94%
苦兒流浪記 93%
祝你忌日快樂 91%
艾莉塔：戰鬥天使 95%
馴龍高手3 94%
幸福綠皮書 94%
上映中的電影:                          ch_name                               eng_name  \
movie_NO.                                                                 
1                           小小夜曲             Little Nights, Little Love   
2          BanG Dream！ FILM LIVE                   BanG Dream！FILM LIVE   
3                            許怨房                               The Room   
4                       82年生的金智英                Kim Ji-Young Born, 1982   
5                          改革好萊塢                This Changes Everything   
6                        女神們的下午茶                    Nothing Like a Dame   
7               

# 無註解程式碼

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

url='https://movies.yahoo.com.tw/movie_intheaters.html'

def yahoo_movie(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html5lib')
    rows = soup.find_all('div', 'release_info_text')
    movies = []
    for row in rows:
        movie = dict()
        movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        pattern = '\d+-\d+-\d+'
        match = re.search(pattern,row.find('div', 'release_movie_time').text)
        movie['release_date'] = match.group(0)
        movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        movie['url'] = row.find('div', 'release_movie_name').a['href']
        movie['movie_id'] =movie['url'].split('.html')[0].split('-')[-1]
        movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        movie['intro'] = row.find('div', 'release_text').text.replace('\n', '').strip()
        trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
        movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        movies.append(movie)
    return movies
yahoo_movie(url)

In [ ]:
url='https://movies.yahoo.com.tw/movie_intheaters.html'

#4.自動儲存所有新上映的電影內容(2頁以上)
def repeat_yahoo_movie(url):
    movies=[]
    movie=yahoo_movie(url)
    movies += movie
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html5lib')
    
    if soup.find('li', 'nexttxt').a['href']:
        nextpage = soup.find('li', 'nexttxt').a['href']
    else:
        nextpage = []

    while nextpage!=[]:
        new_movie=yahoo_movie(nextpage)
        movies += new_movie
        resp = requests.get(nextpage)
        soup = BeautifulSoup(resp.text, 'html5lib')
        if soup.find('li', 'nexttxt').a:
            nextpage = soup.find('li', 'nexttxt').a['href']
        else:
            nextpage = []
            
    return movies
        
repeat_yahoo_movie(url)

In [ ]:
import pandas as pd
import numpy as np

data=yahoo_movie(url)
name=['ch_name','eng_name','release_date','expectation','url','movie_id','poster_url','intro','trailer_url']
test=pd.DataFrame(columns=name,data=data)
test.index = np.arange(1,len(test)+1)
test.index.names = ['movie_NO.']

print('目前上映', len(data), '部電影')

print('期待值> 90% :' )
n='90%'
for a in data:
    if a['expectation'] > n :
        print(a['ch_name'],a['expectation'])
        
print('上映中的電影:',test)

test.to_csv('newmovie.csv',encoding='utf_8_sig')